# Projetinho de implementação do algoritmo KNN

Autores: Renata Leite e César Augusto

Dataset utilizado: https://archive-beta.ics.uci.edu/dataset/19/car+evaluation

## Introdução

In [ ]:
# Importando as bibliotecas necessárias:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter

In [ ]:
# Lendo o dataset e nomeando cada coluna:
names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'evaluation']
ds = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data", names=names)

In [ ]:
ds.head()

,buying,maint,doors,persons,lug_boot,safety,evaluation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


Veja abaixo as features que compõem o dataset:

- buying:	preço de compra;

- maint: preço de manutenção;

- doors: número de portas;

- persons: capacidade de pessoas;

- lug_boot: tamanho do porta-malas;

- safety: segurança do carro;

- evaluation: feature target, informa a avaliação das condições do carro para compra, dividindo-se em: "unacc" (70.023 %), "acc" (22.222 %), "good" (3.993 %) e "v-good" (3.762 %).


## Pré-processamento dos dados

Vamos iniciar a parte de pré-processamento dos dados:

In [ ]:
# Utilizando a função LabelEncoder() para transformar cada rótulo em um número inteiro exclusivo, atribuido com base na ordem alfabética:
lb_enc = LabelEncoder()
ds['buying'] = lb_enc.fit_transform(ds['buying'])
ds['maint'] = lb_enc.fit_transform(ds['maint'])
ds['lug_boot'] = lb_enc.fit_transform(ds['lug_boot'])
ds['safety'] = lb_enc.fit_transform(ds['safety'])

# Utilizando a função .replace() para transformar strings em números inteiros:
ds['doors'].replace(['2', '3', '4', '5more'], [2, 3, 4, 5], inplace=True)
ds['persons'].replace(['2', '4', 'more'], [2, 4, 5], inplace=True)

In [ ]:
# Separando em dados com label ("vizinhos" que serão averiguados) e sem label (aqueles que queremos classificar):
label, nolabel = train_test_split(ds, test_size = 0.3)

In [ ]:
# Excluindo as labels do conjunto de dados que será posteriormente localizado entre seus vizinhos e classificado:
y_nolabel = nolabel['evaluation']
nolabel = nolabel.drop('evaluation', axis = 1)

## Implementação do algoritmo KNN

Imagine que cada coluna é equivalente a uma coordenada que ajuda a localizar o nosso ponto em um espaço multidimensional. Quanto mais variáveis independentes usadas, maior o número de dimensões situando o nosso ponto. A ideia da implementação do KNN seguirá, portanto, as seguintes etapas:

- Primeiro, criaremos uma função que descobre a distância euclidiana entre um ponto num espaço multidimensional e um outro ponto a sua volta. Caso o conceito de "distância euclidiana" seja novo para você, pense no teorema de pitágoras: se existem dois pontos em um plano cartesiano e queremos encontrar a menor linha reta que os separa, estamos atrás, na verdade, da hipotenusa de um triângulo retângulo. No nosso caso será a mesma coisa, com a diferença que, para encontrarmos a distância geométrica num espaço multidimensional, teremos que aplicar o teorema de pitágoras repetidas vezes, de acordo com o número de dimensões existentes.

- Em seguida, com base na função elaborada no passo anterior, criaremos uma outra função para compara todas as distâncias entre um ponto e seus vizinhos e definir quais são seus K vizinhos mais próximos.

- Por último, faremos a predição final com base nos K vizinhos mais próximos.

In [ ]:
def calcula_distancia (a, b):
  """ Calcula a distância euclidiana entre dois vetores.
  Entrada: um vetor 'a' e um vetor 'b'.
  Saída: a distância euclidiana (um único número).   """
  # Definindo a quantidade de características a serem contadas no dataset, também chamada de "dimensão", e a variável 'distancia', inicialmente nula:
  dim = len(a)
  distancia = 0

  # Calculando, para todos os valores de dimensão, a distância entre os dois pontos - aplicação da fórmula de distância euclidiana:
  for i in range(dim):
    distancia += (abs(a[i] - b[i]))**2
  distancia = (distancia)**(1/2)    # Raiz quadrada

  return distancia

In [ ]:
def busca_vizinhos (label, nolabel_row, k):
  """ Encontra os k vizinhos mais próximos do ponto.
  Entrada: Um conjunto de dados com label, um único
  ponto (vetor de features) sem label para ser clas-
  sificado e um número de vizinhos k.
  Saída: um vetor com todas as distâncias.       """
  # Definindo a variavel 'distancias', inicialmente uma lista vazia:
  distancias = list()

  # Encontrando as distancias entre o ponto dado e todos os seus vizinhos do conjunto de dados:
  for row in label:
    distancia = calcula_distancia(nolabel_row, row[:6])    # Aqui, estamos deixando de lado a coluna referente à label (ou row[7])
    distancias.append((row[-1], distancia))
    distancias.sort(key=lambda tup: tup[1])    # Ordena as distâncias

  # Definindo a variavel 'vizinhos', inicialmente uma lista vazia:
  vizinhos = list()
  
  # Encontrando quais são os vizinhos mais próximos (menores distâncias):
  for i in range(k):
    vizinhos.append(distancias[i][0])

  return vizinhos

In [ ]:
def faz_predicao(ponto, vizinhos, k):
  """ Prediz a label de um ponto com base nos K vizinhos mais próximos.
  Entrada: Um ponto (vetor) a ser classificado, um conjunto de dados com
  label e um número k que determina a quantidade de vizinhos que serão 
  utilizados para a predição.
  Saída: a label predita para o ponto fornecido.                     """
  # Definindo a variável 'vizinhos' com base na função anterior e criando a lista 'labels', inicialmente vazia:
  vizinhos = busca_vizinhos(vizinhos, ponto, k)
  labels = list()

  # Adicionando as labels dos vizinhos mais próximos a lista 'labels':
  for vizinho in vizinhos:
    labels.append(vizinho)
  
  # Encontrando o valor mais frequente da lista de labels:
  ocorrencias = Counter(labels)
  label = ocorrencias.most_common(1)[0][0]
  
  return label

## Verificação do algoritmo criado

In [ ]:
# Estabelecendo os valores do nosso dataset:
vizinhos = label.values.tolist()
pontos = nolabel.values.tolist()
k = 6

labels_previstas = list()

# Adicionando as labels previstas à lista 'labels_previstas' de maneira ordenada.
for ponto in pontos:
  labels_previstas.append(faz_predicao(ponto, vizinhos, k))

print('Os valores previstos para as labels de cada ponto são:', labels_previstas)

Os valores previstos para as labels de cada ponto são: ['acc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'vgood', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'acc', 'acc', 'unacc', 'unacc', 'acc', 'acc', 'unacc', 'acc', 'vgood', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'acc', 'unacc', 'acc', 'unacc', 'unacc', 'good', 'good', 'unacc', 'vgood', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'good', 'acc', 'unacc', 'acc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'un

In [ ]:
labels_esperadas = y_nolabel.values.tolist()

i = 0
acertos = 0
erros = 0
while i != len(labels_esperadas):
  if labels_esperadas[i] == labels_previstas[i]:
    acertos += 1
  else:
    erros += 1
  i += 1

acuracia = (acertos / (acertos + erros)) * 100
print(acuracia)

92.48554913294798


Referências utilizadas:

- https://realpython.com/knn-python/

- https://towardsdatascience.com/how-to-build-knn-from-scratch-in-python-5e22b8920bd2

- https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/

- https://medium.com/sanrusha-consultancy/k-nearest-neighbor-classification-and-vehicle-evaluation-d1d693e10ad1